In [1]:
"""Process the patch files generated by extractPatches.py and prepare
them for analysis by Kmeans and diffusion-maps"""

from glob import glob
import pickle as pk
from os import system
import numpy as np
from os.path import isdir
from minio import Minio

def pad_patch(patch):
    too_big=True
    size=patch.shape[0]
    for sz_block in size_tresholds:
        if size<sz_block:
            too_big=False
            break
    if too_big:
        return None,size

    pad=np.zeros([sz_block,sz_block],dtype=np.float16)
    _from=int((sz_block-size)/2)
    _to = int(sz_block-_from)
    # print(size,sz_block,_from,_to)
    pad[_from:_to,_from:_to]=patch
    return pad,sz_block
    
def extract_vectors():
    Vec_dict={}
    for sz in size_tresholds:
        Vec_dict[sz]=[]
    
    for filename in glob('%s/tiles/*extracted*'%local_data):
        #print('filename=',filename)
        D=pk.load(open(filename,'rb'))
        for e in D:
            patch=np.array(e['normalized_patch'],dtype=np.float16)
            padded,sz=pad_patch(patch)
            if not padded is None:
                Vec_dict[sz].append(padded)
    #return a dict of arrays partitioned according to size.
    return Vec_dict

def run(cmd):
    print('cmd=',cmd)
    return system(cmd)


In [21]:
def get_client(creds_file):
    with open(creds_file) as f:
        creds = json.load(f)
    return Minio(
        's3.amazonaws.com',
        secure=True, **creds)



In [11]:
bucket_name='mousebraindata-open'

In [6]:
tgzfile_name=patch_objects[0].object_name
tgzfile_name

'MD657/MD657-N1-2017.02.27-15.52.27_MD657_2_0002_lossless_patches.tgz'

In [16]:
s3_name = 's3://%s'%tgzfile_name
out_name='patches2.tgz'
command='aws s3 cp s3://%s %s'%(tgzfile_name,out_name)
command


'aws s3 cp s3://MD657/MD657-N1-2017.02.27-15.52.27_MD657_2_0002_lossless_patches.tgz patches2.tgz'

In [18]:
%%time
run(command)

cmd= aws s3 cp s3://MD657/MD657-N1-2017.02.27-15.52.27_MD657_2_0002_lossless_patches.tgz patches2.tgz
CPU times: user 1.37 ms, sys: 1.58 ms, total: 2.95 ms
Wall time: 610 ms


256

In [20]:
%pwd ../../../data/

'/Users/yoavfreund/projects/shapeology/shapeology_code/notebooks/AWSmanagement'

In [23]:
creds_file='/Users/yoavfreund/VaultBrain/s3-creds.json'
#local_data='/dev/shm/data'
local_data='/Users/yoavfreund/projects/shapeology/data/'
#exec_dir='/home/ubuntu/shapeology_code/scripts'
exec_dir='/Users/yoavfreund/projects/shapeology/scripts'
bucket_name='mousebraindata-open'
sub_dir='MD657/'
size_tresholds=[15,27,51,81,151,201]


client = get_client(creds_file)

objects = client.list_objects(bucket_name, prefix=sub_dir)
patch_objects=[]
for object in objects:
    name=object.object_name
    if 'lossless_patches' in name and '-N' in name:
        #print(name)
        patch_objects.append(object)

In [ ]:
i=0
f=0
N=0
total_MB=0
s3_path='s3://%s/%s/'%(bucket_name,sub_dir)
for object in patch_objects:
    ALL_V={}
    
    tgz_file=object.object_name
    run('rm %s/tiles/*'%local_data)
    'aws s3 cp %s/%s/%s'%(s3_path,tgzfile_name,out_name,)
    run('aws s3 cp s3://%s%s %s/%s'%(stack,bucket,patch_file,local_data,patch_file))
    run('tar xzvf %s/%s -C /'%(local_data,patch_file))
    if isdir(local_data+'/tiles/dev'):
        print('moving from dev')
        run('mv {0}/tiles/dev/shm/data/tiles/ {0}/tmp_tiles/'.format(local_data))
        run('rm -rf {0}/tiles/'.format(local_data))
        run('mv {0}/tmp_tiles/ {0}/tiles/'.format(local_data))
    V=extract_vectors()
    for sz in size_tresholds:
        if len(V[sz])>0:
            Array=np.array(V[sz],dtype=float16)
            print(sz,Array.shape)
            if not sz in ALL_V:
                ALL_V[sz]=Array
            else:
                ALL_V[sz]=concatenate([ALL_V[sz],Array])
    pk.dump(ALL_V,open(local_data+'/'+patch_file+'.pkl','wb'))